In [16]:
import cv2
import numpy as np
import os
import SimpleITK as itk
import pydicom
import shutil

In [5]:
def image_erode(image, size):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
    image = cv2.erode(image, kernel)
    return image


def image_expand(image, size):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
    image = cv2.dilate(image, kernel)
    return image

'''
    此ｂｌｏｃｋ封装一些常见的操作
'''
# 读取单个DICOM文件
def read_dicom_file(file_name):
    header = pydicom.read_file(file_name)
    image = header.pixel_array
    image = header.RescaleSlope * image + header.RescaleIntercept
    return image
# 读取mhd文件
def read_mhd_image(file_path):
    header = itk.ReadImage(file_path)
    image = itk.GetArrayFromImage(header)
    return np.array(image)


# 保存mhd文件
def save_mhd_image(image, file_name):
    header = itk.GetImageFromArray(image)
    itk.WriteImage(header, file_name)

In [15]:
def ExtractSLROI(sl_path, roi_save_dir):
    sl_names = os.listdir(sl_path)
    for sl_name in sl_names:
        print '-'*15, 'processing %s' % sl_name, '-'*15
        roi_save_path = os.path.join(roi_save_dir, sl_name)
        if not os.path.exists(roi_save_path):
            os.mkdir(roi_save_path)
        for phase in ['NC', 'ART', 'PV']:
            image = read_mhd_image(os.path.join(sl_path, sl_name, phase+'_Image.mhd'))
            mask = read_mhd_image(os.path.join(sl_path, sl_name, phase+'_Mask.mhd'))
            # image[mask == 0] = 0
            [ys, xs] = np.where(mask != 0)
            border = 5
            roi_image = image[np.min(ys) - border:np.max(ys)+border, np.min(xs)-border: np.max(xs)+border]
            save_path = os.path.join(roi_save_path, phase+'_ROI.mhd')
            save_mhd_image(roi_image, save_path)

# 将非ROI的区域置为0
def ExtractSLROIZero(sl_path, roi_save_dir):
    sl_names = os.listdir(sl_path)
    for sl_name in sl_names:
        print '-'*15, 'processing %s' % sl_name, '-'*15
        roi_save_path = os.path.join(roi_save_dir, sl_name)
        if not os.path.exists(roi_save_path):
            os.mkdir(roi_save_path)
        for phase in ['NC', 'ART', 'PV']:
            image = read_mhd_image(os.path.join(sl_path, sl_name, phase+'_Image.mhd'))
            mask = read_mhd_image(os.path.join(sl_path, sl_name, phase+'_Mask.mhd'))
            image[mask == 0] = 0
            [ys, xs] = np.where(mask != 0)
            border = 0
            roi_image = image[np.min(ys) - border:np.max(ys)+border, np.min(xs)-border: np.max(xs)+border]
            save_path = os.path.join(roi_save_path, phase+'_ROI.mhd')
            save_mhd_image(roi_image, save_path)

# 将非RO的区域置为0I，这c里还会提取Expand之后的ROI
def ExtractSLExpandROIZero(sl_path, roi_save_dir):
    sl_names = os.listdir(sl_path)
    for sl_name in sl_names:
        print '-'*15, 'processing %s' % sl_name, '-'*15
        roi_save_path = os.path.join(roi_save_dir, sl_name)
        if not os.path.exists(roi_save_path):
            os.mkdir(roi_save_path)
        for phase in ['NC', 'ART', 'PV']:
            image = read_mhd_image(os.path.join(sl_path, sl_name, 'SignificantLayer', phase+'_Image.mhd'))[0]
            mask = read_mhd_image(os.path.join(sl_path, sl_name, 'SignificantLayer',  phase+'_Mask.mhd'))[0]
            expand_mask = read_mhd_image(os.path.join(sl_path, sl_name, 'SignificantLayer', phase+'_Mask_Expand.mhd'))[0]
            from copy import copy
            copy_image = copy(image)
            # print np.shape(copy_image)
            # print np.shape(expand_mask)
            copy_image[expand_mask == 0] = 0
            image[mask == 0] = 0
            [ys, xs] = np.where(mask != 0)
            border = 0
            roi_image = image[np.min(ys) - border:np.max(ys)+border, np.min(xs)-border: np.max(xs)+border]
            [ys, xs] = np.where(expand_mask != 0)
            roi_image_expand = copy_image[np.min(ys) - border:np.max(ys)+border, np.min(xs)-border: np.max(xs)+border]
            save_path = os.path.join(roi_save_path, phase+'_ROI.mhd')
            save_expand_path = os.path.join(roi_save_path, phase+'_ROI_Expand.mhd')
            save_mhd_image(roi_image, save_path)
            save_mhd_image(roi_image_expand, save_expand_path)

def ExtractSLROIS(sl_paths, roi_save_dir):
    states = ['category_by_level']
    types = ['3', '4', '5']
    for state in states:
        for cur_type in types:
            cur_sl_path = os.path.join(sl_paths, state, cur_type)
            roi_save_path = os.path.join(roi_save_dir, state, cur_type)
            ExtractSLExpandROIZero(cur_sl_path, roi_save_path)
            # show_ROI_size(cur_sl_path)

def show_ROI_size(sl_path):
    sl_names = os.listdir(sl_path)
    for sl_name in sl_names:
        size = []
        print '-'*15, 'processing %s' % sl_name, '-'*15
        for phase in ['NC', 'ART', 'PV']:
            mask = read_mhd_image(os.path.join(sl_path, sl_name, phase+'_Mask.mhd'))
            # image[mask == 0] = 0
            [ys, xs] = np.where(mask != 0)
            cur_size = [np.max(ys) - np.min(ys), np.max(xs) - np.min(xs)]
            size.append(cur_size)
        print sl_name, '->', size

ExtractSLROIS(
    '/home/give/Documents/dataset/LI-RADS/data/whole_data/',
    '/home/give/Documents/dataset/LI-RADS/data/roi_data/'
)

--------------- processing 3580198-3151190 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3382520-2823271 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 1506046-3188371 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3322971-2727042 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 2136190-2784390 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3303654-2708475 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 2987694-2276963 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 2322808-2322808 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)


--------------- processing 3318667-2720750 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3372433-2806976 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 1464234-1951494 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3196596-2558917 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3256760-2792594 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3212586-2583630 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 8075216-2801504 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3223398-2592323 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)


(512, 512)
(512, 512)
--------------- processing 8158985-3199789 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3631930-3242626 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 2527988-1733972 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3747409-3398197 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3258825-2636730 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3527796-3062127 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3721834-3369717 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3950729-3693551 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)


(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 2970918-2246507 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 2796945-2036079 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3187050-2543573 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 8152526-3177106 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 2810769-2050257 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3085944-2406122 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 3078732-2397303 ---------------
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
--------------- processing 8130824-3132064 ---------------
(512, 512)
(512, 512)


In [17]:
def convert_format(roi_dir):
    '''
        将我们上述提取的ROI转化为代码相应的格式
    '''
    for level in [3, 4, 5]:
        cur_dir = os.path.join(roi_dir, str(level))
        for name in os.listdir(cur_dir):
            source = os.path.join(
                cur_dir, name
            )
            target = os.path.join(
                roi_dir, name+'_'+str(level)
            )
            shutil.copytree(
                source,
                target
            )

In [18]:
convert_format(
    '/home/give/Documents/dataset/LI-RADS/data/roi_data/category_by_level'
)